In [12]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [13]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'd-vp-model-without-NumMos'
id_name = "Id"
target = 'WnvPresent'
nummos_col = "NumMosquitos"

file_name = "./d-vp/test_West-Nile Virus Prediction.csv"
test_file_name = "./d-vp/test_West-Nile-Virus-Prediction-without-nummos-no-label.csv"

kaggle_file_name = "./d-vp/kaggle-test-West Nile Virus Prediction.csv"
kaggle_test_file_name = "./d-vp/kaggle-test-West Nile Virus Prediction-without-nummos-no-label.csv"
kaggle_pred_file_name = "./d-vp/kaggle-test-West Nile Virus Prediction-without-nummos-predictions.csv"

In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)

df = df.drop([target, nummos_col],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

         Date                                            Address  \
0  2009-09-17       South Brandon Avenue, Chicago, IL 60617, USA   
1  2011-08-19  2800 North Francisco Avenue, Chicago, IL 60618...   
2  2007-08-24   8000 South Kedzie Avenue, Chicago, IL 60652, USA   
3  2007-06-29       4200 West 127th Street, Alsip, IL 60803, USA   
4  2007-07-25        South Stony Island Avenue, Chicago, IL, USA   

                  Species  Block               Street  Trap  \
0  CULEX PIPIENS/RESTUANS     13            S BRANDON  T209   
1           CULEX PIPIENS     28      N FRANCISCO AVE  T147   
2  CULEX PIPIENS/RESTUANS     80         S KEDZIE AVE  T072   
3          CULEX RESTUANS     42           W 127TH PL  T135   
4           CULEX PIPIENS     10   S STONY ISLAND AVE  T138   

                  AddressNumberAndStreet   Latitude  Longitude  \
0           1300  S BRANDON, Chicago, IL  41.740641 -87.546587   
1     2800  N FRANCISCO AVE, Chicago, IL  41.932094 -87.700117   
2        8000 

In [15]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)    


In [16]:
print(pred_label[:10])
print(pred_neg_prpb[:10])
print(pred_pos_prpb[:10])

import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print(np.mean(y_true==y_pred))

from sklearn import metrics
#fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
#metrics.auc(fpr, tpr)
metrics.roc_auc_score(y_true, pred_pos_prpb)

[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
[0.9969771758187562, 0.6697038114070892, 0.5650212466716766, 0.9988932299893349, 0.9667103327810764, 0.18216443061828613, 0.505611926317215, 0.9988576810574159, 0.4585828185081482, 0.8184832483530045]
[0.0030228241812437773, 0.33029618859291077, 0.43497875332832336, 0.001106770010665059, 0.03328966721892357, 0.8178355693817139, 0.49438807368278503, 0.0011423189425840974, 0.5414171814918518, 0.18151675164699554]
0.8867745004757374


0.8996293128029655

In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)
kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

         Date                                            Address  \
0  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
1  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
2  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
3  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
4  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   

                  Species  Block           Street  Trap  \
0  CULEX PIPIENS/RESTUANS     41   N OAK PARK AVE  T002   
1          CULEX RESTUANS     41   N OAK PARK AVE  T002   
2           CULEX PIPIENS     41   N OAK PARK AVE  T002   
3        CULEX SALINARIUS     41   N OAK PARK AVE  T002   
4         CULEX TERRITANS     41   N OAK PARK AVE  T002   

              AddressNumberAndStreet  Latitude  Longitude  AddressAccuracy  
0  4100  N OAK PARK AVE, Chicago, IL  41.95469 -87.800991                9  
1  4100  N OAK PARK AVE, Chicago, IL  41.95469 -87.800991                9  
2  41

In [23]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [25]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

   Id  WnvPresent
0   1           0
1   2           0
2   3           0
3   4           0
4   5           0
(116293, 2)


In [26]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '4a1d8524-4cf0-4e02-a22d-ade6bfb9ada1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4a1d8524-4cf0-4e02-a22d-ade6bfb9ada1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 13:27:53 GMT'},
  'RetryAttempts': 0}}